In [1]:
import pandas as pd
from z3 import *
from time import time
from copy import copy

floors = set(range(4, 22))

male_floors = set([8, 10, 16, 19])
female_floors = set([7, 14, 18, 20])
mixed_floors = floors.difference(male_floors).difference(female_floors)

rf_floors = set(range(5, 22, 4))
laundry_floors = set([9, 17])

freshmen_floors = set([6, 8, 11, 14, 16, 17, 20])
senior_floors = floors.difference(freshmen_floors)

corridor_nonaircon_pos = set(map(str, range(101, 107) + range(115, 120)))
corridor_aircon_pos = set(map(str, range(109, 115)))
suite_room_letters = map(lambda x: chr(ord('A') + x), range(6))
suite_nonaircon_pos = set([str(suite_num) + letter for suite_num in [100, 107] for letter in suite_room_letters])
suite_aircon_pos = set([str(suite_num) + letter for suite_num in [108] for letter in suite_room_letters])
aircon_pos = corridor_aircon_pos.union(suite_aircon_pos)
nonaircon_pos = corridor_nonaircon_pos.union(suite_nonaircon_pos)
corridor_pos = corridor_nonaircon_pos.union(corridor_aircon_pos)
suite_room_pos = suite_nonaircon_pos.union(suite_aircon_pos)
absent_laundry_pos = set(map(str, range(103, 107)))
room_pos = corridor_pos.union(suite_room_pos)

def get_valid_floors(is_male, pref_mixed, is_freshmen):
    gender_floors = male_floors if is_male else female_floors
    gender_floors = gender_floors.union(mixed_floors) if pref_mixed else gender_floors
    seniority_floors = freshmen_floors if is_freshmen else senior_floors
    return gender_floors.intersection(seniority_floors)

def get_valid_pos(floor_num, is_suite, is_aircon):
    pos = room_pos
    pos = pos.difference(corridor_pos) if is_suite else pos.difference(suite_room_pos)
    pos = pos.difference(nonaircon_pos) if is_aircon else pos.difference(aircon_pos)
        
    if floor_num not in floors:
        pos = pos.difference(room_pos)
        
    if floor_num in rf_floors:
        pos = pos.difference(suite_aircon_pos)
    
    if floor_num in laundry_floors:
        pos = pos.difference(absent_laundry_pos)

    return pos

def get_valid_rooms(is_male=True, pref_mixed=False, is_freshmen=False, is_suite=True, is_aircon=False):
    valid_floors = get_valid_floors(is_male, pref_mixed, is_freshmen)
    valid_rooms = [str(floor_num) + '-' + pos for floor_num in valid_floors for pos in get_valid_pos(floor_num, is_suite, is_aircon)]    
    return set(valid_rooms)

def get_all_rooms():
    rooms = []
    for floor_num in floors:
        if floor_num in laundry_floors:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos.difference(absent_laundry_pos)])
        elif floor_num in rf_floors:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos.difference(suite_aircon_pos)])
        else:
            rooms.extend([str(floor_num) + '-' + pos for pos in room_pos])
    return rooms
    
class Person():
    def __init__(self, name, gender, pref_mixed, usp_status, is_suite, is_aircon, floor_pref, room_pref):
        self.name = name
        self.gender = gender
        self.pref_mixed = pref_mixed
        self.usp_status = usp_status
        self.is_suite = is_suite
        self.is_aircon = is_aircon
        self.floor_pref = floor_pref
        self.room_pref = room_pref
    
    def __str__(self):
        return self.name
    
    def valid_pos(self):
        return get_valid_pos(16,
                             self.is_suite == "suite",
                             self.is_aircon == "aircon"
                            )
        
    def valid_floors(self):
        return get_valid_floors(self.gender == "male",
                               self.pref_mixed == "yes",
                               self.usp_status == "freshmen"
                               )
    
    def valid_rooms(self):
        return get_valid_rooms(self.gender == "male",
                               self.pref_mixed == "yes",
                               self.usp_status == "freshmen",
                               self.is_suite == "suite",
                               self.is_aircon == "aircon"
                              )
    
    __repr__ = __str__

In [2]:
class Person:
    def __init__(self, entry_id, gender, person_type):
        self.entry_id = entry_id
        self.gender = gender
        self.person_type = person_type

In [3]:
class Room:
    def __init__(self, level, position):
        self.level = level
        self.position = position
        self.room_type = RoomType(self.is_suite(), self.is_aircon())
        
    def __init__(self, room_string):
        self.level = room_string.split('-')[0]
        self.position = room_string.split('-')[1]
        self.room_type = RoomType(self.is_suite(), self.is_aircon())

        
    def __str__(self):
        return str(self.level) + '-' + str(self.position)

    def is_aircon(self):
        return self.position in aircon_pos
    
    def is_suite(self):
        return self.position in suite_room_pos
    
    def get_type(self):
        return self.room_type.get_attributes()

    def get_inverse_type(self):
        return self.room_type.get_inverse_attributes()


class RoomType:
    def __init__(self, is_suite, is_aircon):
        self.is_suite = is_suite
        self.is_aircon = is_aircon
        
    # Initialises from description Eg. USP, Single (6 bdrm Apt, Non Air-Con)
    @staticmethod 
    def make_room_type(description):
        is_suite = '6 bdrm Apt' in description
        is_aircon = 'Non Air-Con' not in description
        return RoomType(is_suite, is_aircon)
        

    def __str__(self):
        return 'Suite: {}, Aircon: {} '.format(self.is_suite, self.is_aircon)
    
    def get_attributes(self):
        suite_string = ''
        aircon_string = ''
        
        if self.is_suite:
            suite_string = 'suite'
        else:
            suite_string = 'non-suite'

        if self.is_aircon:
            aircon_string = 'aircon'
        else:
            aircon_string = 'non-aircon'

        return [suite_string, aircon_string]
    
    def get_inverse_attributes(self):
        suite_string = ''
        aircon_string = ''
        
        if self.is_suite:
            suite_string = 'non-suite'
        else:
            suite_string = 'suite'

        if self.is_aircon:
            aircon_string = 'non-aircon'
        else:
            aircon_string = 'aircon'

        return [suite_string, aircon_string]



In [4]:
k = Room('4-100C')
print k.get_type()
print k.get_inverse_type()

['suite', 'non-aircon']
['non-suite', 'aircon']


In [5]:
def solve(nrows=None):
    input_data_frame = pd.read_csv('data/usprc.csv', nrows=nrows, dtype=object).fillna('').values
    constraints = []
    standard_implications = []
    preferences = []
    opt = Optimize()

    # Populate people
    all_people = []
    for row in input_data_frame:
        person = Person(*row[:3])
        all_people.append(person)
        
        # Populate First Preferences
        preferred_type = RoomType.make_room_type(row[3])
        
        for room_attribute in preferred_type.get_attributes():
            preference_implication = Bool("{} gets {}".format(person.entry_id, room_attribute))
            preferences.append(preference_implication)

       
    # Populate rooms
    all_rooms = map(Room, get_all_rooms())
        
        
    # Add obvious implcations
    for person in all_people:
        gets_suite = Bool("{} gets {} room".format(person.entry_id, 'suite'))
        gets_non_suite = Bool("{} gets {} room".format(person.entry_id, 'non-suite'))
        
        standard_implications.append(Or(gets_suite, gets_non_suite))

        gets_aircon = Bool("{} gets {} room".format(person.entry_id, 'aircon'))
        gets_non_aircon = Bool("{} gets {} room".format(person.entry_id, 'non-aircon'))
        
        standard_implications.append(Or(gets_aircon, gets_non_aircon))
    
    
    
    # Populate all room possible assignments
    assignments = {}
    for person in all_people:
        assignments[person] = {}

        for room in all_rooms:
            possibility = Bool("{} gets {}".format(person.entry_id, room))
            
            # Add possibility to possible assignments
            assignments[person][room] = possibility
            
            # Add the possibility's implication (i.e the person gets this room type etc..)
            for attribute in room.get_type():
                implication = Bool("{} gets {} room".format(person.entry_id, attribute))
                standard_implications.append(Implies(possibility, implication))

            for attribute in room.get_inverse_type():
                inverse_implication = Bool("{} gets {} room".format(person.entry_id, attribute))
                standard_implications.append(Implies(possibility, Not(inverse_implication)))
     
    # One person can only get one room
    for person in all_people:
        constraints.append(Or(assignments[person].values()))

    # One room can only be assigned to one person
    for person in all_people:
        for room in all_rooms:
            for other_person in all_people:
                if other_person == person:
                    continue
            
                implication = Implies(assignments[person][room], Not(assignments[other_person][room]))
                constraints.append(implication)
            
    # Solve
    for constraint in constraints:
        opt.add(constraint)
    
    for implication in standard_implications:
        opt.add(implication)
    
    for preference in preferences:
        opt.add(preference)
        
        
    opt.check()
    model = opt.model()
    
    all_symbols = [assignments[person][room] for person in all_people for room in all_rooms]
    return filter(lambda x: is_true(model.eval(x)), all_symbols)

In [7]:
%time solve(10)

CPU times: user 9.02 s, sys: 61 ms, total: 9.08 s
Wall time: 9.11 s


[132083 gets 7-108C,
 133621 gets 7-108E,
 131858 gets 7-108F,
 105327 gets 11-108A,
 167885 gets 6-108D,
 167320 gets 15-108B,
 168255 gets 16-108E,
 171415 gets 11-108E,
 170839 gets 4-108D,
 101335 gets 20-108C]

In [174]:
pd.read_csv('data/usprc.csv', nrows=10, dtype=object).fillna('')

,Entry ID,Gender Description,Classification Description 9,Room Preference Description 1,Room Preference Description 2,Room Preference Description 3,Room Preference Comments,Floor Gender Preference,Nationality Description 4,PR Status,Admit Term,Enrollment Term,Faculty,Medical,Enrollment Status
0,132083,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (6 bdrm Apt, Non Air-Con)",15100,Mixed Gender Floor,SINGAPORE CITIZEN,FALSE,1510,4,ARTS & SOCIAL SCIENCES,,CURRENT
1,133621,Female,USP Senior UG,"USP, Single (Corridor, Non Air-Con)","USP, Single (6 bdrm Apt, Non Air-Con)",,#18-103 or #07-103. facing CAPT.,Single Gender Floor,SINGAPORE CITIZEN,FALSE,1510,4,SCIENCE,,CURRENT
2,131858,Female,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)",,#18-110,Single Gender Floor,SINGAPORE CITIZEN,FALSE,1510,4,ARTS & SOCIAL SCIENCES,Respiratory/breathing difficulties if the air ...,CURRENT
3,105327,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)",,,#19-113 my current room,Single Gender Floor,Indonesian,FALSE,1410,6,SCIENCE,,CURRENT
4,167885,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
5,167320,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Non Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
6,168255,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Non Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
7,171415,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (6 bdrm Apt, Air-Con)","USP, Single (Corridor, Non Air-Con)",#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
8,170839,Female,USP Senior UG,"USP, Single (6 bdrm Apt, Non Air-Con)","USP, Single (Corridor, Non Air-Con)",,#feefivefuudfam,No Preference,SINGAPORE CITIZEN,FALSE,1610,2,ARTS & SOCIAL SCIENCES,,CURRENT
9,101335,Male,USP Senior UG,"USP, Single (Corridor, Air-Con)","USP, Single (6 bdrm Apt, Air-Con)",,#greenhousebesthouse OR #seniorfloor15,Mixed Gender Floor,SINGAPORE CITIZEN,FALSE,1410,6,ENGINEERING,Nil,CURRENT
